In [49]:
import numpy as np
import pandas as pd

from lingualytics.preprocessing import remove_lessthan, remove_punctuation, remove_stopwords
from lingualytics.stopwords import en_stopwords
from texthero.preprocessing import remove_digits

from sentence_transformers import SentenceTransformer


from torch.nn import CosineSimilarity
import torch
import pickle
print("all libraries satisfied")

all libraries satisfied


In [50]:
DATASET = "faqs2.xlsx"
df=pd.read_excel(DATASET)
df

,Q,A
0,What is kandi?,kandi (pronounced kan·dee) is a platform that ...
1,Have feedback or want to know more?,We are a passionate set of application focused...
2,What components does kandi cover?,kandi helps you select software components acr...
3,How do I use kandi?,kandi provides two simplified experiences to h...
4,How do I shortlist components on kandi?,You can use the below filters to shortlist com...
5,How do I implement the components that I have ...,The component listing and detailed insights pa...


In [51]:
df['procd_Q'] = df['Q'].pipe(remove_digits).pipe(remove_punctuation)#.pipe(remove_lessthan,length=3)\
                                                    #.pipe(remove_stopwords,stopwords=en_stopwords.union(hi_stopwords))
df

,Q,A,procd_Q
0,What is kandi?,kandi (pronounced kan·dee) is a platform that ...,What is kandi
1,Have feedback or want to know more?,We are a passionate set of application focused...,Have feedback or want to know more
2,What components does kandi cover?,kandi helps you select software components acr...,What components does kandi cover
3,How do I use kandi?,kandi provides two simplified experiences to h...,How do I use kandi
4,How do I shortlist components on kandi?,You can use the below filters to shortlist com...,How do I shortlist components on kandi
5,How do I implement the components that I have ...,The component listing and detailed insights pa...,How do I implement the components that I have ...


In [52]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [53]:
q_embs = model.encode(df['procd_Q'], convert_to_tensor=True)

C:\Users\pc\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


In [37]:
q_embs.shape

torch.Size([6, 384])

In [38]:
usr_query = "tell me about kandi"

In [39]:
df_usr_query = pd.DataFrame([usr_query], columns=["usr_query"])
df_usr_query

,usr_query
0,tell me about kandi


In [54]:
df_usr_query['clean_usr_Q'] = df_usr_query['usr_query'].pipe(remove_digits) \
                                    .pipe(remove_punctuation) 
df_usr_query


,usr_query,clean_usr_Q
0,tell me about kandi,tell me about kandi


In [55]:
usr_q_emb = model.encode(df_usr_query['clean_usr_Q'], convert_to_tensor=True)
usr_q_emb.shape

C:\Users\pc\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


torch.Size([1, 384])

In [42]:
import torch

In [43]:
cos_fn = torch.nn.CosineSimilarity(dim=1,eps=1e-6)
idx = np.argmax(cos_fn(usr_q_emb,q_embs)).item()

In [44]:
df['A'][idx]

'kandi (pronounced kan·dee) is a platform that helps developers pick the right library, package, code samples, APIs, and cloud functions, by analyzing over 430 million knowledge items.'

In [46]:

def pred_answer(usr_query):
    df_usr_query = pd.DataFrame([usr_query], columns=["usr_query"])
    df_usr_query['clean_usr_Q'] = df_usr_query['usr_query'].pipe(remove_digits) \
                                    .pipe(remove_punctuation) 
    usr_q_emb = model.encode(df_usr_query['clean_usr_Q'], convert_to_tensor=True)
    cos_fn = torch.nn.CosineSimilarity(dim=1,eps=1e-6)
    idx = np.argmax(cos_fn(usr_q_emb,q_embs)).item()
    return df['A'][idx]

In [47]:
while True:
    usr_query = input("Ask a query: ")
    if usr_query == "exit":
        break
    else:
        print(pred_answer(usr_query))
        print("-----------------------------------------------")

Ask a query: tell me about kandi


C:\Users\pc\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


kandi (pronounced kan·dee) is a platform that helps developers pick the right library, package, code samples, APIs, and cloud functions, by analyzing over 430 million knowledge items.
-----------------------------------------------
Ask a query: i want to know more
We are a passionate set of application focused techies. We’d love to hear from you on your feedback, questions, and any other comments.
Direct Message us on Twitter Message @OpenWeaverInc
You can email us at kandi.support@openweaver.com
Join our Discord community here
-----------------------------------------------
Ask a query: components in kandi
kandi helps you select software components across:
Packages from all package managers and repositories
Source Code across all major code repositories
Cloud Functions and APIs across all hyperscale cloud providers
-----------------------------------------------
Ask a query: how can i implement the components in kandi?
The component listing and detailed insights page have links to the